In [13]:
import acopy
print(dir(acopy))


['Ant', 'Colony', 'Solution', 'Solver', 'SolverPlugin', '__author__', '__builtins__', '__cached__', '__doc__', '__email__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'ant', 'plugins', 'solvers', 'utils']


In [12]:
pip show acopy


Name: acopy
Version: 0.7.0
Summary: Ant Colony Optimization for Tthe Traveling Salesman Problem.
Home-page: https://github.com/rhgrant10/acopy
Author: Robert Grant
Author-email: rhgrant10@gmail.com
License: Apache Software License 2.0
Location: c:\users\yoga\appdata\local\programs\python\python39\lib\site-packages
Requires: click, networkx, tsplib95
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [11]:
colony = acopy.Colony()
solver = acopy.Solver()
graph = acopy.Graph()


AttributeError: module 'acopy' has no attribute 'Graph'

In [6]:
pip install acopy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For neural networks
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# For Ant Colony Optimization
from acopy import Ant, AntColony, Graph


ImportError: cannot import name 'AntColony' from 'acopy' (c:\Users\Yoga\AppData\Local\Programs\Python\Python39\lib\site-packages\acopy\__init__.py)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# Parameter ranges
T_min, T_max = 50, 150          # Temperature in °C
P_min, P_max = 1, 10            # Pressure in atm
C_A_min, C_A_max = 0.1, 1.0     # Concentration of A in mol/L
C_B_min, C_B_max = 0.1, 1.0     # Concentration of B in mol/L
S_min, S_max = 100, 500         # Agitation Speed in rpm


In [ ]:
# Number of samples
n_samples = 1000

# Generate random samples within the parameter ranges
T = np.random.uniform(T_min, T_max, n_samples)
P = np.random.uniform(P_min, P_max, n_samples)
C_A = np.random.uniform(C_A_min, C_A_max, n_samples)
C_B = np.random.uniform(C_B_min, C_B_max, n_samples)
S = np.random.uniform(S_min, S_max, n_samples)

# Simulate yield based on a synthetic function
# The function includes nonlinear interactions and noise
def simulate_yield(T, P, C_A, C_B, S):
    # Convert temperature to Kelvin
    T_K = T + 273.15
    # Simulated yield function
    yield_percent = (
        (C_A * C_B * np.exp(-5000 / T_K)) * 
        (1 + 0.1 * np.sin(P)) * 
        (1 + 0.05 * np.log(S)) +
        np.random.normal(0, 0.5, len(T))
    )
    # Ensure yield is between 0% and 100%
    yield_percent = np.clip(yield_percent, 0, 100)
    return yield_percent

# Calculate yield
Yield = simulate_yield(T, P, C_A, C_B, S)


In [ ]:
# Create a DataFrame
data = pd.DataFrame({
    'Temperature': T,
    'Pressure': P,
    'Concentration_A': C_A,
    'Concentration_B': C_B,
    'Agitation_Speed': S,
    'Yield': Yield
})

# Display first few rows
print(data.head())


In [ ]:
# Summary statistics
print(data.describe())


In [ ]:
# Compute correlation matrix
corr_matrix = data.corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Pairplot to visualize relationships
sns.pairplot(data, diag_kind='kde')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Features and target variable
X = data.drop('Yield', axis=1)
y = data['Yield']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Build the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=1
)


In [ ]:
# Plot loss over epochs
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training History')
plt.show()


In [ ]:
# Evaluate on test data
test_loss = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Test MSE: {test_loss:.4f}')


In [ ]:
# Predict on test data
y_pred = model.predict(X_test_scaled)


In [ ]:
# Scatter plot of actual vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel('Actual Yield')
plt.ylabel('Predicted Yield')
plt.title('Actual vs Predicted Yield')
# Line showing perfect predictions
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.show()


In [ ]:
# Define bounds for each parameter
bounds = {
    'Temperature': (T_min, T_max),
    'Pressure': (P_min, P_max),
    'Concentration_A': (C_A_min, C_A_max),
    'Concentration_B': (C_B_min, C_B_max),
    'Agitation_Speed': (S_min, S_max)
}


In [ ]:
def objective_function(params):
    # Convert params dict to array
    param_values = np.array([
        params['Temperature'],
        params['Pressure'],
        params['Concentration_A'],
        params['Concentration_B'],
        params['Agitation_Speed']
    ]).reshape(1, -1)
    
    # Scale the parameters
    param_values_scaled = scaler.transform(param_values)
    
    # Predict yield using the neural network
    predicted_yield = model.predict(param_values_scaled)[0][0]
    
    # Since ACO minimizes the objective, we return the negative yield
    return -predicted_yield


In [ ]:
from scipy.optimize import differential_evolution

# Define bounds in the format required by differential_evolution
bounds_list = [
    (T_min, T_max),          # Temperature
    (P_min, P_max),          # Pressure
    (C_A_min, C_A_max),      # Concentration A
    (C_B_min, C_B_max),      # Concentration B
    (S_min, S_max)           # Agitation Speed
]

# Objective function for differential evolution
def de_objective(x):
    params = {
        'Temperature': x[0],
        'Pressure': x[1],
        'Concentration_A': x[2],
        'Concentration_B': x[3],
        'Agitation_Speed': x[4]
    }
    return objective_function(params)

# Run the optimization
result = differential_evolution(
    de_objective,
    bounds_list,
    strategy='best1bin',
    maxiter=100,
    popsize=15,
    tol=1e-6,
    mutation=(0.5, 1),
    recombination=0.7,
    seed=42
)

# Optimal parameters
optimal_params = result.x
optimal_yield = -result.fun

print('Optimal Parameters:')
print(f'Temperature: {optimal_params[0]:.2f} °C')
print(f'Pressure: {optimal_params[1]:.2f} atm')
print(f'Concentration_A: {optimal_params[2]:.2f} mol/L')
print(f'Concentration_B: {optimal_params[3]:.2f} mol/L')
print(f'Agitation_Speed: {optimal_params[4]:.2f} rpm')
print(f'Predicted Maximum Yield: {optimal_yield:.2f}%')


In [ ]:
# Maximum yield in the initial dataset
max_initial_yield = data['Yield'].max()
print(f'Maximum Yield in Initial Data: {max_initial_yield:.2f}%")


In [ ]:
# Convergence plot
plt.figure(figsize=(8, 6))
plt.plot(result.convergence)
plt.xlabel('Iterations')
plt.ylabel('Objective Function Value')
plt.title('Optimization Convergence')
plt.show()
